In [ ]:
# Inputs

INPUT_FILE = 'datas/Borsdata_2021-01-04.xlsx'

MONEY_TO_USE = 130000
NO_COMPS_TO_BUY = 25

In [ ]:
# Import pandas and ignore warnings

import pandas as pd
pd.set_option('mode.chained_assignment', None)

In [ ]:
# Read Excel file and sort by company name

df = pd.read_excel(INPUT_FILE,header=[0,1],index_col=0)
df.sort_index(inplace=True)

In [ ]:
# Apply filter: enterprise value > 1000 MSEK with positive momentums and trends

ev1000 = df['Börsvärde']['Senaste'] > 1000
utv3 = df['Kursutveck.']['Utveck.  3m'] > 0
utv6 = df['Kursutveck.']['Utveck.  6m'] > 0
utv12 = df['Kursutveck.']['Utveck.  1 år'] > 0
utv6vs3 = df['Kursutveck.']['Utveck.  6m'] > df['Kursutveck.']['Utveck.  3m']
utv12vs6 = df['Kursutveck.']['Utveck.  1 år'] > df['Kursutveck.']['Utveck.  6m']
ma50 = df['Kurs / MA']['MA 50d'] > 0
ma200 = df['Kurs / MA']['MA 200d'] > 0
ma200vs50 = df['Kurs / MA']['MA 200d'] > df['Kurs / MA']['MA 50d']

df_filt = df[ev1000 & utv3 & utv6 & utv12 & utv6vs3 & utv12vs6 & ma50 & ma200 & ma200vs50]

In [ ]:
# Add momentum ranks and sort by total rank

header = pd.MultiIndex.from_product([['Momentum rank'],
                                     ['Rank 3m','Rank 6m','Rank 1 år','Tot']])
header2 = pd.MultiIndex.from_product([['Kursutveck.'],
                                     ['Utveck.  3m','Utveck.  6m','Utveck.  1 år']])

for i in range(3):
    df_filt[header[i]] = df_filt[header2[i]].rank(ascending=False,method='first').astype(int)
df_filt[header[3]] = (df_filt[header[0]] + df_filt[header[1]] + df_filt[header[2]]).rank(ascending=True,method='first').astype(int)
df_filt.sort_values(by=[header[3]],inplace=True)

In [ ]:
# Display amount of stocks to buy for each company

money_per_stock = MONEY_TO_USE / NO_COMPS_TO_BUY
df_filt['Antal aktier'] = round(money_per_stock / df_filt['Aktiekurs']['Senaste']).astype(int)
df_filt.head(NO_COMPS_TO_BUY)

In [ ]:
# Display number of companies from each list

stock_lists = ['Large Cap', 'Mid Cap', 'Small Cap', 'First North']
stock_lists_dict = {}

for stock_list in stock_lists:
    stock_lists_dict[stock_list] = sum(df_filt.head(NO_COMPS_TO_BUY)['Info']['Lista'] == stock_list)

for stock_list,num in stock_lists_dict.items():
    print(f'{stock_list:11} {num}')